In [66]:
import pandas as pd

data = pd.read_csv('train.csv')

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer


In [68]:
for col in data.columns:
    if data[col].isnull().any()==True and (data[col].dtype=='float64'):
        median = data[col].median()
        data[col] = data[col].fillna(median)  

In [69]:
#drop the na's
data.dropna(axis=0, subset=['SalePrice'], inplace=True)

y=data.SalePrice
# X= data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

X = data.drop(['SalePrice'], axis=1)

one_hot_features = pd.get_dummies(X)

#train_X, test_X, train_y, test_y = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)

#imp = Imputer()
#train_X = imp.fit_transform(one_hot_features)
#test_X = imp.fit_transform(test_X)



In [70]:
test = pd.read_csv('test.csv')

#replace the float column NAs with the median value
for col in test.columns:
    if test[col].isnull().any()==True and (data[col].dtype=='float64'):
        median = data[col].median()
        data[col] = data[col].fillna(median)  

        
#X2 = imp.fit_transform(test)

one_hot_features2 = pd.get_dummies(test)

final_train, final_test = one_hot_features.align(one_hot_features2, join='left', axis=1)


In [71]:
#predictions = model.predict(test_X)

# from sklearn.metrics import mean_absolute_error
# print("MAE is "+ str(mean_absolute_error(predictions, test_y)))

from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=124, learning_rate=0.05)
model.fit(final_train, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=124,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [72]:
#drop any NAs that are left
#test.dropna(axis=0, subset=['SalePrice'], inplace=True)

In [73]:

predictions = model.predict(final_test)



In [74]:
#save the predictions as a dataframe and then write it to CSV format
predictions_file = pd.DataFrame({
    'Id': test['Id'],
    'SalePrice': predictions
    })

predictions_file.to_csv('submission_xgb.csv', index=False)